In [4]:
import numpy as np

# Data Preparation

In [5]:
import data_module as dm
DLS_X, DLS_Y, DLS_W, DLD_X, DLD_Y, DLD_W = dm.import_data()
#DLS = data large scale, contains thousands of datapoints
#DLD = data low dimension, contains less than or hundreds of datapoints

In [6]:
#select DLD first, as we gonna try to build up the model (faster execution)
data_w = DLD_W[0] #maximum weight
data_x = DLD_X[0] #datapoints
data_y = DLD_Y[0] #optimum values

# Solution Representation
The solution is represented as boolean mask of $N$ size, where $N$ is the size of data. 1 or True means the consecutive data is selected / carried away. 0 or False menas the consecutive data is not selected/carried away

In [7]:
sol = np.random.random(len(data_x))
sol

array([0.26883136, 0.58847016, 0.33888147, 0.60731337, 0.86338164,
       0.93504649, 0.74749793, 0.72260778, 0.7234924 , 0.28433166,
       0.5799968 , 0.96382489, 0.52424693, 0.27918827, 0.25235398,
       0.09644272, 0.00990284, 0.4418656 , 0.31693068, 0.5861497 ])

In [8]:
#after discretizatoin
np.round(sol)

array([0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1.])

In [9]:
idx = np.arange(len(data_x))
idx = idx[np.round(sol).astype(bool)]
carried = data_x.iloc[idx]
carried

,weight,price
2,72,83
4,46,4
5,55,44
6,8,6
7,35,82
8,75,92
9,61,25
11,77,56
12,40,18
13,63,58


# Fitness Function

In [10]:
def fit(sol, data, max_w) :
    #Discretization
    mask = np.round(sol).astype(bool)
    
    #decode
    idx = np.arange(len(data))
    idx = idx[mask]
    data_solution = data.iloc[idx]
    
    #price 
    price = np.sum(data_solution['price'])
    weight = np.sum(data_solution['weight'])
    if weight <= max_w:
        return price
    else :
        return 0

In [11]:
# Hyper parameters 
fmin = 0
fmax = 1
A = 1
alpha = 0.98
gamma = 0.98
population = 20

In [12]:
# def init_x (population, data_size):
#     return np.random.random((population,data_size))

# def init_f (population, fmin,fmax):
#     return  np.random.uniform(fmin,fmax,population)

# def init_v (poulation):
#     return np.zeros(population)

# def init_y (solutions,data):
#     #solutions : list of solution
#     Y = np.zeros(len(solutions))
#     for i,sol in enumerate(solutions) : 
#         Y[i] = fit(sol,data)
#     return Y

# def init_r (population):
#     return np.random.random(population)

# def init_A (population):
#     # Aku lupa A itu apa
#     return None

In [13]:
class bat:
    def __init__(self, population, data, max_w, fmin, fmax, A, alpha, gamma):
        self.population = population
        self.data = data
        self.max_w = max_w
        self.fmin = fmin
        self.fmax = fmax
        self.A = A
        self.alpha = alpha
        self.gamma = gamma
        self.data_size = len(data)
        self.best_sol = None
        
        self.init_x()
        self.init_f()
        self.init_v()
        self.init_y()
        self.init_r()
                
    def init_x (self):
        self.solutions =  np.random.random((self.population,self.data_size))
        
    def init_f (self):
        self.f = np.random.uniform(self.fmin,self.fmax,self.population)

    def init_v (self):
        self.v = np.zeros(self.population)

    def init_y (self):
        Y = np.zeros(len(self.solutions))
        for i,sol in enumerate(self.solutions) : 
            Y[i] = fit(sol,self.data, self.max_w)
        
        self.Y = Y

    def init_r (self):
        self.r = np.random.random(self.population)
        
    
    def update_f(self):
        self.fmin = np.min(self.f)
        self.fmax = np.max(self.f)
        betha = np.random.random(len(self.f))
        self.f = betha*(self.fmax-self.fmin) + self.fmin
    
    def update_v(self):
        self.find_best_solution()

        self.v = self.v + self.f * (self.solutions - self.best_sol)
    
    #__________________
    def find_best_solution(self):
        self.best_sol = self.solutions[np.argmax(self.Y)]

In [14]:
baat = bat(10,data_x,data_w,fmin,fmax,A,alpha,gamma)

In [15]:
# def train(bat, epoch):
#     for i in range(epoch):
#         #update kelelawar
        

# ini line coba

In [16]:
baat.Y

array([636., 415., 523., 646., 729., 844., 318., 695., 622., 524.])

In [17]:
np.argmax(baat.Y)

5

In [18]:
baat.find_best_solution()

In [19]:
baat.best_sol

array([0.76491008, 0.62947458, 0.86878549, 0.83726367, 0.00393408,
       0.55970329, 0.58472065, 0.76376327, 0.95907977, 0.57175306,
       0.66044182, 0.85724254, 0.27986824, 0.25053502, 0.56722426,
       0.12430153, 0.96450925, 0.6270931 , 0.4374662 , 0.71896433])

In [21]:
baat.find_best_solution()
baat.best_sol

array([0.76491008, 0.62947458, 0.86878549, 0.83726367, 0.00393408,
       0.55970329, 0.58472065, 0.76376327, 0.95907977, 0.57175306,
       0.66044182, 0.85724254, 0.27986824, 0.25053502, 0.56722426,
       0.12430153, 0.96450925, 0.6270931 , 0.4374662 , 0.71896433])

In [22]:
baat.v

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
baat.v + (baat.f * (baat.solutions - baat.best_sol))

ValueError: operands could not be broadcast together with shapes (10,) (10,20) 

In [24]:
baat.v + 1

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
(baat.solutions - baat.best_sol)

array([[-0.24376635, -0.138716  , -0.42969738, -0.29394839,  0.12292392,
         0.36104628, -0.51352   , -0.63319222, -0.34181968,  0.30713048,
         0.24601974, -0.81440484,  0.49280885,  0.60762916,  0.03226096,
         0.82272088, -0.26781903, -0.35529481, -0.21473772,  0.22836577],
       [-0.14416837, -0.20540689, -0.35325951, -0.62217465,  0.02199252,
        -0.22000711, -0.24969126, -0.31652188, -0.49789349, -0.55507039,
        -0.17432414, -0.68952836,  0.01482615, -0.08240312, -0.48185379,
         0.15761716, -0.43537593,  0.06564099,  0.18628599,  0.23697686],
       [-0.0791834 , -0.40765963, -0.58834714, -0.02187937,  0.7720351 ,
         0.05279792, -0.41048822, -0.10394666, -0.48464581,  0.16189007,
        -0.55233568, -0.82633426, -0.25533639,  0.71201266, -0.38101012,
         0.32317175, -0.12103848,  0.25368948,  0.55964412, -0.36625958],
       [ 0.07763339, -0.13442928, -0.1595214 , -0.14920195,  0.51944976,
        -0.48962162, -0.17774778, -0.13948789, -